In [9]:
from sympy.tensor.array import derive_by_array
import matplotlib.pyplot as plt
import scipy.optimize as opt
import numpy as np
import sympy as sp
import math
from expression import pipe

In [19]:
# mesh = Mesh("mesh_big.xml")
L = 0.003
E = 1.75*1e+11
nu = 0.316
lmbda = (E*nu/((1+nu)*(1-2*nu)))
mu = (E/2/(1+nu))
alpha = 1e-5 # коэффициент теплового расширения
kappa = (alpha/(3*mu + 2*lmbda))
d = 1 # interpolation degree
Vue = VectorElement('CG', mesh.ufl_cell(), d)
# displacement finite element
V = FunctionSpace(mesh, Vue)
dU = TrialFunction(V)
t = 1
a = -1000
dT = a*t*sqrt((x[0] - L/2)*(x[0] - L/2) + (x[1] - L/2)*(x[1] - L/2))
dT010 = Expression('a*t*(x[1] - L/2)/(sqrt((x[0]- L/2)*(x[0] - L/2) + (x[1] - L/2)*(x[1] - L/2)) + 1e-16)',
degree=2, t = t, a = a, L = L)
dT100 = Expression('a*t*(x[0] - L/2)/(sqrt((x[0] -L/2)*(x[0] - L/2) + (x[1] - L/2)*(x[1] - L/2)) + 1e-16)',
degree=2, t = t, a = a, L = L)

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1134, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 311, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "C:\Users\griho\AppData\Roaming\Python\Python310\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2062, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "C:\Users\griho\AppData\Roaming\Python\Python310\site-packages\debugpy\_vendored\pydevd\pydevd.py", line 2098, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
def boundary_D(x, on_boundary):
    L = 0.003
    tol = 1e-16
    return on_boundary and (near(0, x[1], tol)
    or near(L, x[1], tol) or near(x[0], L, tol) )
def boundary(x, on_boundary):
    return on_boundary

In [ ]:
def eps(v):
    return sym(grad(v))
def strain2voigt(e):
    return as_vector([e[0,0],e[1,1],2*e[0,1]])
def voigt2stress(s):
    return as_tensor([[s[0], s[2]],
[s[2], s[1]]])
def sigma(v):
    return voigt2stress(dot(C, strain2voigt(eps(v))))

In [ ]:
sigma_xx = Expression('lmbda*(1 - nu)/nu*(eps_xx) + lmbda*(eps_yy - alpha*dT) ', degree=2, nu = nu, 
    alpha = alpha, dT = dT, lmbda = lmbda, eps_xx = eps_xx, eps_yy = eps_yy)
sigma_yy = Expression('lmbda*(1 - nu)/nu*(eps_yy) +lmbda*(eps_xx - alpha*dT) ', degree=2, nu = nu,
alpha = alpha, dT = dT, lmbda = lmbda, eps_xx = eps_xx, eps_yy = eps_yy)
sigma_xy = Expression('2*mu*eps_xy ', degree=2, E = E, nu = nu, alpha = alpha, t = t, dT = dT, eps_xy = eps_xy, mu = mu)

In [ ]:
g5 = Expression(('-sigma_xx ’’’, ’’’ -sigma_xy '), degree=2, sigma_xx = sigma_xx, sigma_xy = sigma_xy,
sigma_yy = sigma_yy )

In [ ]:
U = Function(V)
aa = inner(sigma(dU), eps(U_))*dx

In [ ]:
CalphadT = Expression(('(lmbda + lmbda*(1 - nu)/nu)*alpha*dT',
'(lmbda + lmbda*(1 - nu)/nu)*alpha*dT'),
degree=2, kappa = kappa, dT = dT, lmbda = lmbda,
nu=nu, mu=mu, alpha = alpha)
L = dot(b, U_)*dx + dot(CalphadT, U_)*dx
+ dot(g5, U_)*ds

In [ ]:
#finding the displacements
solve(aa == L, U, bc, solver_parameters=
{'linear_solver':'umfpack',
'preconditioner':'ilu'})

In [ ]:
#finding the stresses
vertex_values_CalphadT = interpolate(CalphadT , V)
stress = sigma(U)

In [ ]:
error_L2 = errornorm(U_D, U, 'L2', mesh = mesh)/(norm(U_D, 'L2', mesh) + 1e-16)
vertex_values_U_D = U_D.compute_vertex_values(mesh)
vertex_values_U = U.compute_vertex_values(mesh)
error_max = np.max(np.abs(vertex_values_U_D - vertex_values_U))/np.max(np.abs(vertex_values_U_D))
print(error_L2, error_max)